In [29]:
import tensorflow as tf 
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.layers import Dense,Flatten,MaxPool2D,Conv2D,Dropout
import matplotlib.pyplot as plt

In [30]:
tf

<module 'tensorflow' from 'c:\\Users\\Akash Ghimire\\anaconda3\\envs\\vision\\lib\\site-packages\\tensorflow\\__init__.py'>

In [31]:
(ds_train_x,ds_train_y),(ds_test_x,ds_test_y)=load_data()

ds_train=tf.data.Dataset.from_tensor_slices((ds_train_x,ds_train_y))
ds_test=tf.data.Dataset.from_tensor_slices((ds_test_x,ds_test_y))

print('len of train_ds :',len(ds_train))
print('len of test_ds :',len(ds_test))

len of train_ds : 60000
len of test_ds : 10000


In [32]:
def preprocess(x,y):
    print('x.shape :',x.shape)

    x=tf.cast(x,tf.float32)
    x=x/255.0
    _,h,w=x.shape
   
    x=tf.reshape(x,[-1,h,w,1])
    
    y=tf.one_hot(y,depth=10)
    return x,y

In [33]:
batch_size=256
ds_train=ds_train.cache().shuffle(buffer_size=2000).batch(batch_size).map(preprocess).prefetch(tf.data.AUTOTUNE)
ds_test=ds_test.cache().batch(batch_size).map(preprocess).prefetch(tf.data.AUTOTUNE)





x.shape : (None, 28, 28)
x.shape : (None, 28, 28)


In [34]:
def get_model():

    inputs=tf.keras.Input(shape=(28,28,1))
    x=Conv2D(64,(5,5),activation='relu')(inputs)
    x=MaxPool2D()(x)
    x=Conv2D(128,(3,3),activation='relu')(x)
    x=MaxPool2D()(x)
    
    x=Flatten()(x)
    x=Dropout(0.4)(x)
    x=Dense(10,activation='softmax')(x)

    model=tf.keras.Model(inputs,x,name='okok')
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [35]:
model=get_model()

In [36]:
model.summary()

Model: "okok"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 64)        1664      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 128)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 3200)              0      

In [37]:
history=model.fit(ds_train,verbose=True,epochs=30,shuffle=True)

Epoch 1/30
235/235 [==============================] - 3s 10ms/step - loss: 0.3099 - accuracy: 0.9118
Epoch 2/30
235/235 [==============================] - 2s 10ms/step - loss: 0.0826 - accuracy: 0.9754
Epoch 3/30
235/235 [==============================] - 2s 10ms/step - loss: 0.0601 - accuracy: 0.9823
Epoch 4/30
235/235 [==============================] - 2s 10ms/step - loss: 0.0493 - accuracy: 0.9853
Epoch 5/30
235/235 [==============================] - 2s 10ms/step - loss: 0.0425 - accuracy: 0.9870
Epoch 6/30
235/235 [==============================] - 2s 10ms/step - loss: 0.0366 - accuracy: 0.9886
Epoch 7/30
235/235 [==============================] - 2s 9ms/step - loss: 0.0325 - accuracy: 0.9896
Epoch 8/30
235/235 [==============================] - 2s 9ms/step - loss: 0.0287 - accuracy: 0.9909
Epoch 9/30
235/235 [==============================] - 2s 9ms/step - loss: 0.0267 - accuracy: 0.9917
Epoch 10/30
235/235 [==============================] - 2s 10ms/step - loss: 0.0232 - accuracy:

In [38]:
model.evaluate(ds_test,verbose=1)

40/40 [==============================] - 1s 8ms/step - loss: 0.0249 - accuracy: 0.9925


[0.024859467521309853, 0.9925000071525574]

In [39]:
predicted=model.predict(ds_test)

40/40 [==============================] - 0s 4ms/step


In [40]:
predicted=tf.argmax(predicted,axis=1)

In [41]:
true_label=tf.argmax(list(ds_test.map(lambda x,y:y).unbatch().as_numpy_iterator()),axis=1).numpy()


In [42]:
from sklearn.metrics import accuracy_score

In [43]:
accuracy_score(true_label,predicted)

0.9925